In [1]:
#notebook to save the injected trials in an array

In [2]:
import csky as cy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import histlite as hl
import copy

In [3]:
def load_data(dataset='combo'):
    repo = cy.selections.Repository()
    cspec = cy.selections.DNNCascadeDataSpecs.DNNC_12yr                             
    psspec = cy.selections.PSDataSpecs.ps_v4_15yr[3:]
    ana = cy.get_analysis(repo, 'version-004-p03', psspec, 'version-001-p02', cspec)
    return ana

In [4]:
def get_gp_conf(ana, template_str, ): 
        template_repo = cy.selections.Repository(
        local_root='/data/ana/analyses/NuSources/2021_DNNCascade_analyses')
        if template_str == 'pi0':

            # get default gamma
            gamma = 2.7

            template = template_repo.get_template('Fermi-LAT_pi0_map')
            gp_conf = {
                'template': template,
                'flux': cy.hyp.PowerLawFlux(gamma),
                'randomize': ['grl'],
                'fitter_args': dict(gamma=gamma),
                'sigsub': True,
                'update_bg' : True,
                'fast_weight' : False,
                'extra_keep' : ['energy' , 'mjd'],
                'dir' : '/data/user/ssclafani/templates/'
                }
            return gp_conf     

In [6]:
ana = load_data()


Setting up Analysis for:
IC86v4_12yr, DNNCascade_12yr
Setting up IC86v4_12yr...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2016_MC.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2011_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2012_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2013_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2014_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2015_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2016_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2017_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2018_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2019_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2020_exp.npy ...
Reading /data/ana/analyses/ps_tracks/version-004-p03/IC86_2021_exp.npy ...
Reading /data/ana/ana

In [7]:
gp_conf = get_gp_conf(ana, 'pi0')
gp_tr = cy.get_trial_runner(ana=ana, conf=gp_conf)

Reading /data/ana/analyses/NuSources/2021_DNNCascade_analyses/templates/Fermi-LAT_pi0_map.npy ...
<- /data/user/ssclafani/templates//IC86v4_12yr.template.npy    
Restore successful.
<- /data/user/ssclafani/templates//DNNCascade_12yr.template.npy    
Restore successful.


In [15]:
#galactic plane paper flux to ns for this dataset

flux_E2dNdE_at_100TeV = 21.8e-12 #TeV/s/cm^2
gp_tr.to_ns(flux_E2dNdE_at_100TeV, flux=cy.hyp.PowerLawFlux(2.7), E0=100, unit=1e3)

2012.2529763198404

In [17]:
ts = []
cs = []
for i in range(5000):
    if i % 100 == 0:
        print(i, end='\r')
    t = gp_tr.get_one_trial(n_sig = 2012, seed = i) 
    ts.append(t[0][0][1].as_array)
    cs.append(t[0][1][1].as_array)

400

In [18]:
ts_dict = {f'seed_{i}' : ts[i] for i in range(len(ts))}
cs_dict = {f'seed_{i}' : cs[i] for i in range(len(cs))}

In [ ]:
np.save('/data/user/ssclafani/GP_injected_trials/trial_tracks_IC86.npy', ts_dict)
np.save('/data/user/ssclafani/GP_injected_trials/trial_cascades_IC86.npy', cs_dict)